# Get Data From Kaggle

In [ ]:
!pip install opendatasets
!pip install kaggle

In [ ]:
import opendatasets as od


od.download("https://www.kaggle.com/datasets/andrewmvd/face-mask-detection")

# Convert Annotation File Format From 'XML' To 'TXT' For Yolo v5

In [126]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Dataset, random_split
from torchvision.io import read_image

from sklearn.model_selection import train_test_split

import xml.etree.ElementTree as ET
from xml.dom import minidom

from tqdm import tqdm

from PIL import Image, ImageDraw

import numpy as np

import matplotlib.pyplot as plt

random.seed(108)

In [5]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [29]:
def convert_to_yolov5(info_dict, base_dir):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join(base_dir, info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [127]:
class_name_to_id_mapping = {"without_mask": 0,
                            "with_mask": 1,
                            "mask_weared_incorrect": 2}

annotations_path = "/content/face-mask-detection/annotations"
annotations_to_path = "/content/face-mask-detection/yolo_annotations"
images_path = "/content/face-mask-detection/images"
root_dir = "/content/face-mask-detection"

In [35]:
for f in os.listdir(annotations_path):
    if f[-3:] == "xml":
        file_path = os.path.join(annotations_path, f)
        xml_data = extract_info_from_xml(file_path)
        convert_to_yolov5(xml_data, annotations_to_path)
    

##Test The Annotations

In [ ]:
annotations = [os.path.join(annotations_to_path, x) for x in os.listdir(annotations_to_path) if x[-3:] == "txt"]

random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("yolo_annotations", "images").replace("txt", "png")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

# Data Split and Make Train/Val/Test Directory

In [128]:
# Read images and annotations
images_folder_path = os.path.join(root_dir, 'images')
annotation_folder_path = os.path.join(root_dir, "yolo_annotations")

images = [os.path.join(images_folder_path, x) for x in os.listdir(images_folder_path)]
annotations = [os.path.join(annotation_folder_path, x) for x in os.listdir(annotation_folder_path) if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [134]:
!mkdir -p images/train -p images/val -p images/test -p annotations/train -p annotations/val -p annotations/test


**Move Files**

In [135]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'annotations/train/')
move_files_to_folder(val_annotations, 'annotations/val/')
move_files_to_folder(test_annotations, 'annotations/test/')

#Install YoloV5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt

In [138]:
!mv annotations labels
!cd ../yolov5 

/bin/bash: line 0: cd: ../yolov5: No such file or directory


##Define Parameters

In [140]:
batch = 20
epochs = 10
cfg = "/content/yolov5/models/yolov5m.yaml"
hyp = ""
name = "/content/runs/train/facemask"
train_imgs_path = "../images/train"
val_imgs_path = "../images/val"
test_imgs_path = "../images/test"
nc = 3
names = ["without_mask", "with_mask", "mask_weared_incorrect"]

In [142]:
import yaml


dict_file = [
    {"path": "/content"},
    {"train": train_imgs_path},
    {"val": val_imgs_path},
    {"test": test_imgs_path},
    {"nc": nc},
    {"names": names}
]
with open("/content/yolov5/data/face_mask_detection.yaml", "w") as file:
    yaml.dump(dict_file, file)

In [158]:
!pip install clearml
!pip install comet_ml

##Train Yolov5s

In [159]:
!python train.py --img 640 --batch 16 --epochs 3 --data face_mask_detection.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=face_mask_detection.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-89-g35d6d9f Python-3.8.10 torch-1.13.1+cu116 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl

##**Inference**

In [161]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source /content/images/test/maksssksksss140.png

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/images/test/maksssksksss140.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-89-g35d6d9f Python-3.8.10 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/images/test/maksssksksss140.png: 448x640 4 with_masks, 256.9ms
Speed: 1.9ms pre-process, 256.9ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2
